# Preparació dades PRA2 Visualització de dades

## raw_Space_Corrected.csv

In [1]:
import pandas as pd
import numpy as np

# Open file1
df_launch = pd.read_csv('raw_Space_Corrected.csv', dtype={'Rocket':np.float64})
print(df_launch.columns)
print(df_launch.shape)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Company Name', 'Location', 'Datum',
       'Detail', 'Status Rocket', ' Rocket', 'Status Mission'],
      dtype='object')
(4324, 9)


In [3]:
# Drop columns
df_launch = df_launch.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

# Country column
df_launch["Country"] = df_launch["Location"].apply(lambda location: location.split(", ")[-1])

# Replace wrong countries
df_launch['Country'] = df_launch['Country'].replace(['Yellow Sea'], 'China')
        # https://en.wikipedia.org/wiki/Yellow_Sea
df_launch['Country'] = df_launch['Country'].replace(['Shahrud Missile Test Site'], 'Iran')
        # https://www.shymkent.info/space/spaceports/shahrud-missile-test-site/
df_launch['Country'] = df_launch['Country'].replace(['Pacific Missile Range Facility'], 'USA')
        # https://en.wikipedia.org/wiki/Pacific_Missile_Range_Facility
df_launch['Country'] = df_launch['Country'].replace(['Gran Canaria'], 'USA')
        # https://nextspaceflight.com/launches/details/228
df_launch['Country'] = df_launch['Country'].replace(['Barents Sea'], 'Russia')
        # https://nextspaceflight.com/launches/details/1344
df_launch['Country'] = df_launch['Country'].replace(['Pacific Ocean'], 'Ukraine')
        # All launches from LP Odyssey (Pacific Ocean) used Zenit-3SL rocket model  
        # https://en.wikipedia.org/wiki/Zenit-3SL
df_launch['Country'] = df_launch['Country'].replace(['New Mexico'], 'USA')
    
# Time info
df_launch['Datum'] = pd.to_datetime(df_launch['Datum'])
df_launch['Year']  = df_launch['Datum'].apply(lambda datetime: datetime.year)
df_launch['Month'] = df_launch['Datum'].apply(lambda datetime: datetime.month)
df_launch['Hour']  = df_launch['Datum'].apply(lambda datetime: datetime.hour)

# Extract Cargo, Model and Serie from Detail
df_launch["Cargo"] = df_launch["Detail"].apply(lambda detail: detail.split(" | ")[1])
df_launch["Model"] = df_launch["Detail"].apply(lambda detail: detail.split(" | ")[0])
df_launch["Serie"] = df_launch["Model"].apply(lambda detail: detail.split(" ")[0])
df_launch["Serie"] = df_launch["Serie"].apply(lambda detail: detail.split("-")[0])
df_launch["Serie"] = df_launch["Serie"].apply(lambda detail: detail.split("/")[0])
df_launch['Serie'] = df_launch['Serie'].replace(["Shtil'"], 'Shtil')
df_launch = df_launch.drop(['Detail'], axis=1)

# Replace some Series
df_launch['Serie'] = df_launch['Serie'].replace(['Commercial'], 'Commercial Titan')
df_launch['Serie'] = df_launch['Serie'].replace(['Black'], 'Black Arrow')
df_launch['Serie'] = df_launch['Serie'].replace(['Blue'], 'Blue Scout')
df_launch['Serie'] = df_launch['Serie'].replace(['Feng'], 'Feng Bao')
df_launch['Serie'] = df_launch['Serie'].replace(['GSLV'], 'GSLV Mk')
df_launch['Serie'] = df_launch['Serie'].replace(['Long'], 'Long March')
df_launch['Serie'] = df_launch['Serie'].replace(['Space'], 'Space Shuttle')

# Replace some Company
df_launch['Company Name'] = df_launch['Company Name'].replace(["Arm??e de l'Air"], "Armè de l'Air")

# Reorder columns
df_launch = df_launch[['Country', 'Location', 'Year', 'Month', 'Hour', 'Datum', 
                     'Company Name', 'Model', 'Serie', 'Cargo', 'Status Mission', ' Rocket', 'Status Rocket']]

df_launch.rename(columns={" Rocket": "Price_launch", "Company Name": "Company"}, inplace=True)

df_launch.head(3)

,Country,Location,Year,Month,Hour,Datum,Company,Model,Serie,Cargo,Status Mission,Price_launch,Status Rocket
0,USA,"LC-39A, Kennedy Space Center, Florida, USA",2020,8,5,2020-08-07 05:12:00+00:00,SpaceX,Falcon 9 Block 5,Falcon,Starlink V1 L9 & BlackSky,Success,50.0,StatusActive
1,China,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020,8,4,2020-08-06 04:01:00+00:00,CASC,Long March 2D,Long March,Gaofen-9 04 & Q-SAT,Success,29.75,StatusActive
2,USA,"Pad A, Boca Chica, Texas, USA",2020,8,23,2020-08-04 23:57:00+00:00,SpaceX,Starship Prototype,Starship,150 Meter Hop,Success,NaN,StatusActive


-------

### raw_all_rockets_from_1957.csv

In [3]:
# Open file
df_rockets = pd.read_csv('raw_all_rockets_from_1957.csv', dtype={'Payload to LEO': float, 'Payload to GTO':float})
print(df_rockets.columns)
print(df_rockets.shape)

Index(['Unnamed: 0', 'Name', 'Cmp', 'Wiki', 'Status', 'Liftoff Thrust',
       'Payload to LEO', 'Stages', 'Strap-ons', 'Rocket Height', 'Price',
       'Payload to GTO', 'Fairing Diameter', 'Fairing Height'],
      dtype='object')
(416, 14)


In [8]:
# Liftoff Thrust
df_rockets['Liftoff Thrust'] = df_rockets['Liftoff Thrust'].str.replace(',','')
df_rockets['Liftoff Thrust'] = df_rockets['Liftoff Thrust'].fillna("0")

# Rocket Height
df_rockets['Rocket Height'] = df_rockets['Rocket Height'].str.replace(' m','')

# Stages
df_rockets['Stages'] = df_rockets['Stages'].apply(str).str.replace('.0','')

# Strap-ons	
df_rockets['Strap-ons'] = df_rockets['Strap-ons'].apply(str).str.replace('.0','')

# Price
df_rockets['Price'] = df_rockets['Price'].str.replace(' million','').str.replace('$','').str.replace(',','')
df_rockets[df_rockets['Price'] == "5,000.0"] = np.nan # https://en.wikipedia.org/wiki/Energia#Development

# Fairing Diameter
df_rockets['Fairing Diameter'] = df_rockets['Fairing Diameter'].str.replace(' m','')

# Fairing Height
df_rockets['Fairing Height'] = df_rockets['Fairing Height'].str.replace(' m','')

df_rockets.rename(columns={"Name": "Model"}, inplace=True)
df_rockets = df_rockets.drop(['Unnamed: 0'], axis=1)

df_rockets.head()

,Model,Cmp,Wiki,Status,Liftoff Thrust,Payload to LEO,Stages,Strap-ons,Rocket Height,Price,Payload to GTO,Fairing Diameter,Fairing Height
0,Tsyklon-3,Yuzhmash,https://en.wikipedia.org/wiki/Tsyklon-3,Retired,3032,4.10,3,0,39.0,NaN,0.00,NaN,NaN
1,Tsyklon-4M,Yuzhnoye,https://en.wikipedia.org/wiki/Cyclone-4M,Planned,3130,5.00,2,0,38.7,45.0,0.91,4.0,NaN
2,Unha-2,KCST,https://en.wikipedia.org/wiki/Unha,Retired,1192,0.20,3,0,28.0,NaN,0.00,NaN,NaN
3,Unha-3,KCST,https://en.wikipedia.org/wiki/Unha,Active,1192,0.20,3,0,32.0,NaN,0.00,1.3,NaN
4,Vanguard,US Navy,https://en.wikipedia.org/wiki/Vanguard_(rocket),Retired,135,0.01,3,0,23.0,NaN,0.00,NaN,NaN


In [9]:
df_rockets.isna().sum()

Model                 0
Cmp                   0
Wiki                  9
Status                0
Liftoff Thrust        0
Payload to LEO        0
Stages                0
Strap-ons             0
Rocket Height        83
Price               310
Payload to GTO        0
Fairing Diameter    185
Fairing Height      249
dtype: int64

## Unió dels dos datasets 

In [10]:
df = pd.merge(left=df_launch, right=df_rockets, left_on='Model', right_on='Model',  how='left')
df.head()

,Country,Location,Year,Month,Hour,Datum,Company,Model,Serie,Cargo,...,Status,Liftoff Thrust,Payload to LEO,Stages,Strap-ons,Rocket Height,Price,Payload to GTO,Fairing Diameter,Fairing Height
0,USA,"LC-39A, Kennedy Space Center, Florida, USA",2020,8,5,2020-08-07 05:12:00+00:00,SpaceX,Falcon 9 Block 5,Falcon,Starlink V1 L9 & BlackSky,...,Active,7607,22.80,2,0,70.0,50.0,8.30,5.2,13.0
1,China,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020,8,4,2020-08-06 04:01:00+00:00,CASC,Long March 2D,Long March,Gaofen-9 04 & Q-SAT,...,Active,2962,3.50,2,0,41.06,29.75,0.00,3.35,10.9
2,USA,"Pad A, Boca Chica, Texas, USA",2020,8,23,2020-08-04 23:57:00+00:00,SpaceX,Starship Prototype,Starship,150 Meter Hop,...,Active,6000,0.00,1,0,50.0,NaN,0.00,NaN,NaN
3,Kazakhstan,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020,7,21,2020-07-30 21:25:00+00:00,Roscosmos,Proton-M/Briz-M,Proton,Ekspress-80 & Ekspress-103,...,Active,10532,0.00,4,0,58.2,65.0,6.00,4.35,15.26
4,USA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020,7,11,2020-07-30 11:50:00+00:00,ULA,Atlas V 541,Atlas,Perseverance,...,Active,10581,17.41,2,4,62.2,145.0,8.29,5.4,26.5


In [11]:
df.columns

Index(['Country', 'Location', 'Year', 'Month', 'Hour', 'Datum', 'Company',
       'Model', 'Serie', 'Cargo', 'Status Mission', 'Price_launch',
       'Status Rocket', 'Cmp', 'Wiki', 'Status', 'Liftoff Thrust',
       'Payload to LEO', 'Stages', 'Strap-ons', 'Rocket Height', 'Price',
       'Payload to GTO', 'Fairing Diameter', 'Fairing Height'],
      dtype='object')

In [12]:
# Reorder columns
df = df[['Country', 'Location', 'Year', 'Month', 'Hour', 'Datum', 'Status Mission', 'Price_launch',
         'Company', 'Model', 'Serie', 'Cargo', 'Price', 'Status Rocket', 'Status', 'Liftoff Thrust', 
         'Stages', 'Strap-ons', 'Rocket Height', 'Fairing Diameter', 'Fairing Height',
         'Payload to LEO', 'Payload to GTO' ,'Wiki']]

# Remove NaN
df = df.loc[pd.notnull(df['Stages'])]
df.isnull().sum()

Country                0
Location               0
Year                   0
Month                  0
Hour                   0
Datum                  0
Status Mission         0
Price_launch        3360
Company                0
Model                  0
Serie                  0
Cargo                  0
Price               3360
Status Rocket          0
Status                 0
Liftoff Thrust         0
Stages                 0
Strap-ons              0
Rocket Height        664
Fairing Diameter    1746
Fairing Height      2879
Payload to LEO         0
Payload to GTO         0
Wiki                  11
dtype: int64

In [14]:
df.dtypes

Country              object
Location             object
Year                  int64
Month                 int64
Hour                  int64
Datum                object
Status Mission       object
Price_launch         object
Company              object
Model                object
Serie                object
Cargo                object
Price                object
Status Rocket        object
Status               object
Liftoff Thrust       object
Stages               object
Strap-ons            object
Rocket Height        object
Fairing Diameter     object
Fairing Height       object
Payload to LEO      float64
Payload to GTO      float64
Wiki                 object
dtype: object

In [ ]:
df.to_csv('launches_rockets.csv', header=True, index=False)